# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.



Описание данных

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита
Необходимо подготовить данные к анализу: выявить аномалии в данных, произвести чистку данных от артефактов и дубликатов и т.д. Необходимо произвести категоризацию данных и выявить факторы, влияющие на погашение кредита в срок.

Шаг 1. Откройте файл с данными и изучите общую информацию.
Вывод
Шаг 2. Предобработка данных
Обработка пропусков
Вывод
Замена типа данных
Вывод
Обработка дубликатов
Вывод
Лемматизация
Вывод
Категоризация данных
Вывод
Шаг 3. Ответьте на вопросы
Шаг 4. Общий вывод

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [4]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head()

In [ ]:
data.info()

Пропущенные значения присутствуют в графах с трудовым стажем и доход в месяц

при выгрузке таблицы мы видим, что в столбцах days_employed  и total_income  есть пропуски.

In [ ]:
display(data.describe()) # сразу видно аномалии

for col in data.columns:
    
    if data[col].dtype == "object":
    
        print(col)
        print(data[col].unique())
        print()

## Шаг 2. Предобработка данных

Обработка пропусков

мы видем что  столбце days_employed  и total_income пропущены значения Nan

In [ ]:
data[data['days_employed'].isnull()].head(10)
data[data['total_income'].isnull()].head(10)

посмотрим количество пропущенных Nan столбце days_employed и total_income

In [ ]:
len(data[data['days_employed'].isna()])

всего пропусков 2174

группировка по столбцу зарплата

In [ ]:
med_salary = data.groupby('income_type')['total_income'].transform('median')


заполним пропуски на медианный доход

In [ ]:
data['total_income'] = data['total_income'].fillna(med_salary)


заменим NAN на среднее значение 

In [ ]:
data.head(10)

просмотрим количество пропусков в таблице

In [ ]:
data['days_employed'].isnull().sum()
print(data.head(10))

In [ ]:
print(data['income_type'].unique())

создадим отдельный дата для каждого типа занятости 

In [ ]:
data_sotrudnik = data[(data['income_type'] == 'сотрудник')]
data_penciober = data[(data['income_type'] == 'пенсионер')]
data_companion = data[(data['income_type'] == 'компаньон')]
data_gosslujaci = data[(data['income_type'] == 'госслужащий')]
data_unemployed = data[(data['income_type'] == 'безработный')]
data_bisnesmen = data[(data['income_type'] == 'предприниматель')]
data_student = data[(data['income_type'] == 'студент')]
data_dikret = data[(data['income_type'] == 'в декрете')]

print(data_sotrudnik.info())
print(data_penciober.info())
print(data_companion.info())
print(data_gosslujaci.info())
print(data_unemployed.info())
print(data_bisnesmen.info())
print(data_student.info())
print(data_dikret.info())




заполняю всю таблицу, так как пропущенные значения есть в этих столбцах, заменим пропущенные значение на 0 все таблицы

In [ ]:
data_sotrudnik.fillna(0)
data_penciober.fillna(0)
data_companion.fillna(0)
data_gosslujaci.fillna(0)
data_unemployed.fillna(0)
data_bisnesmen.fillna(0)
data_student.fillna(0)
data_dikret.fillna(0)

print(data.info())


In [ ]:
# сформируем сводную табличку с группировкой по типу занятости и измерением медианного дохода
group_income = df.groupby('income_type')['total_income'].median()

In [ ]:
# заполним пропуски на медианный доход, основываясь на типе занятости
for i in group_income.index:

    df.loc[(data['income_type'] == i) & (df['total_income'].isnull()), 'total_income'] = group_income[i]

In [ ]:
data.groupby('income_type')['days_employed'].mean() / 365 # смотрю средний стаж по типам занятости, для наглядности поделил на 365, чтобы легче было интерпретировать

In [ ]:
data['days_employed'] = data['days_employed'].apply(lambda x: x / 24 if x > 0 else abs(x))

# проверяем

data.groupby('income_type')['days_employed'].mean() / 365

# картина уже куда лучше, стаж вполне адекватный, теперь мы можем заменить пропуски

data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))



In [ ]:
# проверяем пропуски еще раз

data.isnull().sum() # пропусков нет, ура!

In [ ]:
# Возраст как вариант можно заполнить на основе семейного статуса, потому что как правило люди в разводе \ женатые \ вдовцы
# старше, нежели люди не состоявшие в браке или в гражданском

data['dob_years'] = data.groupby('family_status')['dob_years'].transform(lambda x: x.replace(0, int(x.median())))

# XNa заменяем просто на наиболее популярный пол

data['gender'] = data['gender'].replace('XNA', 'F')

заменяем вещественный тип данных на целочисленный. Переведем данные float в int в столбце total_income 

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')

### Обработка дубликатов

In [ ]:
data.duplicated().sum()

In [ ]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

In [ ]:
print(data['education'].unique())
print()
print('Уникальные значения в графе family_status:')
print(data['family_status'].unique())
print()
print('Уникальные значения в графе gender:')
print(data['gender'].unique())
print()
print('Уникальные значения в графе income_type:')
print(data['income_type'].unique())
print()
print('Уникальные значения в графе purpose:')
print(data['purpose'].unique())
print()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates().reset_index(drop= True)

проверим остались ли дубликаты

In [ ]:
data.duplicated().sum()

### Лемматизация

In [ ]:
# создадим лемматизацию

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
def do_lemma(row):
    lemma = m.lemmatize(row)
    return lemma

дополнительный столбец лемматизации

In [ ]:
data['purpose_lemma'] = data['purpose'].apply(do_lemma)

In [ ]:
созданим страку из уникальных целей 

In [ ]:
all_lemmas_list = data['purpose_lemma'].values
flat_lemmas_list = []
for sublist in all_lemmas_list:
   for item in sublist:
       flat_lemmas_list.append(item)

In [ ]:
посчитаем самые часто встречаемые слова для цели кредита

In [ ]:
print(all_lemmas_list)
from collections import Counter
print(Counter(flat_lemmas_list))

In [ ]:
# созданим словарь с новыми словами и впишем объединеные по значению слова 

In [ ]:
def dictionary(world, data):
    for row in data:
        if world == row:
            return row
dict = []
dict.append(dictionary('жилье', flat_lemmas_list))
dict.append(dictionary('недвижимость', flat_lemmas_list))
dict.append(dictionary('образование', flat_lemmas_list))
dict.append(dictionary('автомобиль', flat_lemmas_list))
dict.append(dictionary('свадьба', flat_lemmas_list))
print(dict)

In [ ]:
#создаем столбец с категориями, которые объединят все значение в одно 

In [ ]:
def category(data):
    if dict[0] in data:
        return dict[0]
    elif dict[1] in data:
        return dict[1]
    elif dict[2] in data:
        return dict[2]
    elif dict[3] in data:
        return dict[3]
    else:
        return dict[4]
    
print()
data['category_purpose'] = data['purpose_lemma'].apply(category)
print(data.loc[:, ['purpose', 'category_purpose']] .head(10))

In [ ]:
data['category_purpose'].value_counts()

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_kindred = data[data['children'] != 0]
data_freekind = data[data['children'] == 0]
#print(df_kindred.info())
kindred_debit = data_kindred['debt'].sum() / len(data_kindred)
freekind_debit = data_freekind['debt'].sum() / len(data_freekind)
print('Процент среди людей с детьми, которые имели задолжности: {:.1%}'.format(kindred_debit))
print('Процент среди бездетных, которые имели задолжности: {:.1%}'.format(freekind_debit))

ВЫВОД :Казалось бы что люди, которые заводят детей более ответственно относятся к выплате долгов, но на деле оказалось наоборот

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
print('Типы семейного положения и их id соответственно:')
print(data['family_status'].unique())
print(data['family_status_id'].unique())
print()
#['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
# 'не женат / не замужем']
#[0 1 2 3 4]

#создаем таблицу data_pivot_family_status где смотрим дожников по категории семейного положения
data_pivot_family_status = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
#считаем процент должников
data_pivot_family_status['ratio %'] = (data_pivot_family_status[1] / data_pivot_family_status[0]) * 100
data_pivot_family_status['ratio %'] = data_pivot_family_status['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_family_status['sum'] = data_pivot_family_status[1] + data_pivot_family_status[0]


print(data_pivot_family_status.loc[:, ['ratio %', 'sum']])
print()
print('Из таблицы видно что те люди кто никогда не был в официальном браке более склонны к задолжностям:')
print('Гражданский брак, не женат / не замужем - 10% с задолжностями')
print('В разводе, вдовец / вдова, женат / замужем - 7-8% с задолжностями')

In [ ]:
fam_pivot = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum'])
fam_pivot.columns = ['all', 'debetors']

# считаем % невозвратов, делим должников на общее число клиентов
fam_pivot['%'] = (fam_pivot['debetors'] / fam_pivot['all']) * 100

display(fam_pivot)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
print('Разобьем доход на уровни: 0, 0-20, 20-50, 50-100, 100-250, 250-500, 500+')

def income(data):
    if data == 0:
        return '0'
    elif data <= 20000:
        return '0-20'
    elif data <= 50000:
        return '20-50'
    elif data <= 100000:
        return '50-100'
    elif data <= 250000:
        return '100-250'
    elif data <= 500000:
        return '250-500'
    else:
        return '500+'
#
data['category_income'] = data['total_income'].apply(income)
print()
#print(data.loc[:, ['total_income', 'category_income']] .head(15))
#
#создаем таблицу data_pivot_income где смотрим дожников по категории заработка
data_pivot_income = data.pivot_table(index='category_income', columns='debt', values='total_income', aggfunc='count')
#считаем процент должников
data_pivot_income['ratio %'] = (data_pivot_income[1] / data_pivot_income[0]) * 100
data_pivot_income['ratio %'] = data_pivot_income['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_income['sum'] = data_pivot_income[1] + data_pivot_income[0]

print(data_pivot_income.loc[:, ['ratio %', 'sum']])
print()
print('Из таблицы видно что те люди кто имеет заработок выше среднего более склонны к задолжностям:')
print('50-250 тысяч рублей в месяц - 9-10% с задолжностями')
print('До 50 и свыше 250 тысяч рублей в месяц - 7% с задолжностями')

Из таблицы видно что те люди кто имеет заработок выше среднего более склонны к задолжностям:
50-250 тысяч рублей в месяц - 9-10% с задолжностями
До 50 и свыше 250 тысяч рублей в месяц - 7% с задолжностями

In [ ]:
data['total_income_category'] = pd.qcut(data['total_income'], 4, labels=["низкий", "средний", "выше среднего", "высокий"])
print(data['total_income_category'].value_counts())


In [ ]:
import plotly.express as px

def pivot(data, index):
 
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum', 'mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', '% невозврата']
    display(pivot.sort_values('% невозврата', ascending = False))
    
    
    
    fig = px.bar(pivot, x = index, y = '% невозврата', color = '% невозврата', title = '% невозврата по ' + index)
    fig.show()
    

pivot(data, 'total_income_category')

Из таблицы видно что те люди кто берет кредит на более мелкие цели более склонны к задолжностям:
Автомобиль, образование, свадьба - 9-11% с задолжностями
Жилье, недвижимость - 7-8% с задолжностями

Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#создаем таблицу data_pivot_income где смотрим дожников по цели кредита
data_pivot_purpose = data.pivot_table(index='category_purpose', columns='debt', values='purpose', aggfunc='count')
#считаем процент должников
data_pivot_purpose['ratio %'] = (data_pivot_purpose[1] / data_pivot_purpose[0]) * 100
data_pivot_purpose['ratio %'] = data_pivot_purpose['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_purpose['sum'] = data_pivot_purpose[1] + data_pivot_purpose[0]

print(data_pivot_purpose.loc[:, ['ratio %', 'sum']])
print()
print('Из таблицы видно что те люди кто берет кредит на более мелкие цели более склонны к задолжностям:')
print('Автомобиль, образование, свадьба - 9-11% с задолжностями')
print('Жилье, недвижимость - 7-8% с задолжностями')



In [ ]:
pivot(data, 'category_purpose')

In [ ]:
round(data.groupby('category_purpose')['debt'].mean() * 100, 2)

## Шаг 4. Общий вывод

Из выше указанных исследований видно что более надежными оказались люди, которые были или состоят в браке, имеют ЗП
до 50 и свыше 250 тысяч рублей в месяц и с более масштабной целью кредита, например жилье или недвижимость